In [17]:
# GodQuestions_Crawling

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from datetime import date

## test_0424

In [79]:
final=pd.DataFrame(columns=["crawling_date","Question_FAR","Answer_FAR","URL_FAR","Question_ENG","Answer_ENG","URL_ENG"])
print(final)

Empty DataFrame
Columns: [crawling_date, Question_FAR, Answer_FAR, URL_FAR, Question_ENG, Answer_ENG, URL_ENG]
Index: []


In [80]:
# 1) main title -> click & title extracting
driver = webdriver.Chrome()
driver.get("https://www.gotquestions.org/Farsi/")

a_tags=driver.find_elements(By.TAG_NAME,"a")
print(len(a_tags))

52


In [81]:
for i in range(1, 52): # 첫 번째 클릭 (대주제) - 1부터 52까지
    

    try:
        element = driver.find_element(By.XPATH, f'/html/body/main/section[1]/div/strong/a[{i}]') # 버튼의 xpath를 이용 (1, 2, 3, ...)
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)

    except NoSuchElementException:
            time.sleep(0.5)
            break

    for ii in range(1, 120): # 두 번째 클릭 (소주제) 

        # 소주제 개수는 대주제마다 달라서, range()로 일단 1부터 120까지 범위 정해놓고
        # 반복문을 실행할 수 없는 상황, 찾으려는 요소가 없어서 더 이상 작업할 수 없을 때 반복문 종료
        try:
            element = driver.find_element(By.XPATH, f'/html/body/main/section[1]/div/strong/a[{ii}]') # 버튼의 xpath를 이용 (1, 2, 3, ...)
            driver.execute_script("arguments[0].click();", element)
            time.sleep(0.5)
        
        # 만약 버튼이 없다면 초기 화면으로 돌아간 후 초기 반복문으로 이동
        except NoSuchElementException:
            driver.get("https://www.gotquestions.org/Farsi/")
            time.sleep(0.5)
            break
            
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # crawling 하는 날짜 date 넣기
        crawling_date = date.today()
        # 제목(소주제)는 <h1></h1> 사이에 존재
        title_far = soup.find('h1').get_text(strip = True)

        # 현재 url
        url_far = driver.current_url
            
        # 답변 내용은 "جواب"과 "English"사이에 존재한다는 공통점
        content_str_far = str(soup.get_text(separator="\n"))
        start_far = content_str_far.find('جواب') + len('جواب') # 답변의 시작 지점
        end_far = content_str_far.find('English') # 답변의 끝 지점
        answer_html_far = content_str_far[start_far : end_far]
        answer_far = BeautifulSoup(answer_html_far, 'html.parser').get_text("\n", strip=True) # <>와 같은 태그 제거하여 텍스트만 남기기
        #answer_kor = re.findall(r'[\uAC00-\uD7A3\s]+', answer_kor) # 한국어 문자와 공백만 추출
        #answer_kor = ''.join(answer_kor)

        # ----------- #

        # English 클릭해서
        #element = driver.find_element(By.XPATH,"/html/body/main/section[1]/div/a")
        
        try :
            element = driver.find_element(By.LINK_TEXT, "English")
            driver.execute_script("arguments[0].click();", element)
            time.sleep(0.5)

        # English 없는 경우 발생...ㅎㄷㄷ
        except NoSuchElementException: 
             # 페르시아어 파일 저장
            new_row = pd.DataFrame({"crawling_date":[crawling_date],
                                    "Question_FAR": [title_far], "Answer_FAR": [answer_far], "URL_FAR": [url_far],
                                    "Question_ENG": "NA", "Answer_ENG": "NA", "URL_ENG": "NA"})
            final = pd.concat([final, new_row], ignore_index=True)
            driver.back()
            time.sleep(0.1)
            break

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
                
        # 영어 내용 크롤링
        # 영어 제목(소주제) <h1></h1> 사이에 존재
        title_eng = soup.find('h1').get_text(strip = True)

        # 현재 url
        url_eng = driver.current_url
                
        # 답변 내용은 "Question"과 "Return to:" 사이에 존재한다는 공통점
        content_str_eng = str(soup.get_text(separator="\n"))
        start_eng = content_str_eng.find('Answer') + len('Answer') # 답변의 시작 지점
        end_eng = content_str_eng.find('Return to:') # 답변의 끝 지점
        answer_html_eng = content_str_eng[start_eng : end_eng]
        answer_eng = BeautifulSoup(answer_html_eng, 'html.parser').get_text("\n", strip=True) # <>와 같은 태그 제거하여 텍스트만 남기기

            
        # 파일 저장
        new_row = pd.DataFrame({"crawling_date":[crawling_date],
                                    "Question_FAR": [title_far], "Answer_FAR": [answer_far], "URL_FAR": [url_far],
                                    "Question_ENG": [title_eng], "Answer_ENG": [answer_eng], "URL_ENG": [url_eng]})
        final = pd.concat([final, new_row], ignore_index=True)
            
        driver.back()
        time.sleep(0.1)

        driver.back()
        time.sleep(0.1)

In [84]:
final

,crawling_date,Question_FAR,Answer_FAR,URL_FAR,Question_ENG,Answer_ENG,URL_ENG
0,2025-04-25,آیا شما حیات جاودانی دارید؟,کتاب مقدس به وضوح راه رسیدن به زندگی جاودانه ر...,https://www.gotquestions.org/Farsi/Farsi-Etern...,Got Eternal Life?,The Bible presents a clear path to eternal lif...,https://www.gotquestions.org/eternal-life.html
1,2025-04-25,چگونه می توان آمُرزش الهی را دریافت نمود؟,"اعمال رسولان فصل 13 آیه 38 به ما می گوید که: ""...",https://www.gotquestions.org/Farsi/Farsi-Got-F...,Got Forgiveness?,"Acts 13:38\n declares, ""Therefore, my brothers...",https://www.gotquestions.org/got-forgiveness.html
2,2025-04-25,من یک مسلمان هستم. چرا باید به فکر مسیحی شدن ب...,مردم معمولا دنباله روی دین و مذهب والدین و یا ...,https://www.gotquestions.org/Farsi/Farsi-Musli...,I am a Muslim. Why should I consider becoming ...,People often follow the religion of their pare...,https://www.gotquestions.org/Muslim-Christian....
3,2025-04-25,"آیا چیزی به عنوان ""زندگی پس از مرگ"" وجود دارد؟",اینکه آیا پس از مرگ زندگی وجود دارد یا نه! یه ...,https://www.gotquestions.org/Farsi/Farsi-life-...,Is there life after death?,"Job, speaking out of his despair, asked, “If a...",https://www.gotquestions.org/is-there-life-aft...
4,2025-04-25,پذیرفتن عیسی مسیح به عنوان نجات دهندۀ شخصی به ...,آیا عیسی مسیح را به عنوان نجات دهندۀ شخصی خود ...,https://www.gotquestions.org/Farsi/Farsi-perso...,What does it mean to accept Jesus as your pers...,Have you accepted Jesus Christ as your persona...,https://www.gotquestions.org/personal-Savior.html
...,...,...,...,...,...,...,...
1547,2025-04-25,ما از زندگی پطرس چه درسی می گیریم؟,شمعون پطرس یا کیفا (صخره) (یوحنا فصل 1 آیه 42)...,https://www.gotquestions.org/Farsi/Farsi-life-...,Who was Peter in the Bible?,"Simon Peter, also known as Cephas (\nJohn 1:42...",https://www.gotquestions.org/life-Peter.html
1548,2025-04-25,فیلیپ در کتاب مقدس چه کسی بود؟,چهار مرد متفاوت در کتاب مقدس وجود دارند که نام...,https://www.gotquestions.org/Farsi/Farsi-Phili...,Who was Philip in the Bible?,There are four different men named Philip ment...,https://www.gotquestions.org/Philip-in-the-Bib...
1549,2025-04-25,ما از زندگی استیفان چه درسی می گیریم؟,اعمال رسولان فصل 6 آیه 5، استیفان را مردی با ا...,https://www.gotquestions.org/Farsi/Farsi-life-...,Who was Stephen in the Bible?,Acts 6:5\n introduces a faithful man of God na...,https://www.gotquestions.org/life-Stephen.html
1550,2025-04-25,ما از زندگی تیموتائوس چه درسی می گیریم؟,تیموتائوس که دریافت کننده دو نامه عهد جدید به ...,https://www.gotquestions.org/Farsi/Farsi-life-...,Who was Timothy in the Bible?,"Timothy, the recipient of the two New Testamen...",https://www.gotquestions.org/life-Timothy.html


In [90]:
final.to_excel(f"/Users/haley/Desktop/2025-1/DS1/code/preprocessing/GodQuestions/GodQuestions_test_Farsi__{crawling_date}.xlsx",index=False)

In [ ]:
# finish !

## 새로운 데이터와 어떻게 병합을 할 것인가?

## split code process

In [20]:
# test
# 1) main title -> click & title extracting
# 2) second titie -> click
# 3) second title & answer -> extracting

In [21]:
# 1) main title -> click & title extracting
driver = webdriver.Chrome()
driver.get("https://www.gotquestions.org/Farsi/")

In [24]:
driver.find_element(By.CSS_SELECTOR,"body > main > section.content-wrap.gradient-to-b > div > strong > a:nth-child(1)").click()

In [25]:
# 2) second titie -> click
elements=driver.find_element(By.CSS_SELECTOR,"body > main > section.content-wrap.gradient-to-b > div > strong > a:nth-child(1)")
PER_URL=elements.get_attribute("href")
elements.click()

In [26]:
# 3) second title & answer -> extracting
response = requests.get(PER_URL)
response.encoding = 'utf-8' 
html_text = response.text


soup=bs(html_text, "html.parser")

# text extrating
content = soup.select_one("div.content")
content.get_text()

'\nسوال\nآیا شما حیات جاودانی دارید؟\n\n\nجواب\nکتاب مقدس به وضوح راه رسیدن به زندگی جاودانه را به ما نشان داده است. نخست، ما باید این حقیقت را بپذیریم که بر ضّد خدا گناه کرده ایم: "زيرا همه گناه كرده\u200cاند و هيچكس به آن كمال مطلوب و پرجلالی كه خدا انتظار دارد، نرسيده است." (رومیان فصل 3 آیه 23). همۀ ما مرتکب اعمالی شده ایم که نه فقط خدا را خشنود نساخته، بلکه ما را سزاوار مجازات نیز نموده اند. از آنجا که همۀ گناهان ما در نهایت بر ضّد خدای ابدی صورت گرفته اند، ما شایستۀ مجازات ابدی هستیم. "زيرا هر كه گناه كند، تنها دستمزدی كه خواهد يافت، مرگ است؛ اما هر كه به خداوند ما عيسی مسيح ايمان آورد، پاداش او از خدا زندگی جاويد است." (رومیان فصل 6 آیه 23).\n\nاما عیسی مسیح، آن پسر ابدی خدا که بدون گناه بود (اول پطرس فصل 2 آیه 22)، انسان شد (یوحنا فصل 1 آیه های 1 و 14)، و مُرد تا جریمۀ گناهان ما را بپردازد. "اما ببينيد خدا چقدر ما را دوست داشت كه با وجود اينكه گناهكار بوديم، مسيح را فرستاد تا در راه ما فدا شود." (رومیان فصل 5 آیه 8). عیسی مسیح بر روی صلیب جان داد (یوحنا فصل 19 آیه های 31 تا 42)

## loop function

In [27]:
# 1) main title -> click & title extracting
driver = webdriver.Chrome()
driver.get("https://www.gotquestions.org/Farsi/")

In [28]:
content=driver.find_element(By.CLASS_NAME,"content")
a_tags=driver.find_elements(By.TAG_NAME,"a")

In [29]:
len(a_tags)

52

In [30]:
final = []

In [31]:
a_tag = driver.find_elements(By.CSS_SELECTOR, ".content a")
print(len(a_tag))

a_tags=driver.find_elements(By.TAG_NAME,"a")
print(len(a_tags))

32
52


In [32]:
final=[]

In [33]:
for i in range(1, len(a_tags),3):
    
    content=driver.find_element(By.CLASS_NAME,"content")
    a_tags=driver.find_elements(By.TAG_NAME,"a")
    
    # 1)
    driver.find_element(By.CSS_SELECTOR,f"body > main > section.content-wrap.gradient-to-b > div > strong > a:nth-child({i})").click()
    time.sleep(1)

    # 2)
    content2=driver.find_element(By.CLASS_NAME,"content")
    a_tags2=driver.find_elements(By.TAG_NAME,"a")
    for j in range(1,len(a_tags2),3):
        elements=driver.find_element(By.CSS_SELECTOR,f"body > main > section.content-wrap.gradient-to-b > div > strong > a:nth-child({j})")
        PER_URL=elements.get_attribute("href")
        elements.click()
        time.sleep(1)
        
        # 3)
        response = requests.get(PER_URL)
        response.encoding = 'utf-8' 
        html_text = response.text
    
        soup=bs(html_text, "html.parser")
    
        # text extrating
        content = soup.select_one("div.content")
        final.append(content.get_text())

        driver.back()

    driver.back()


final

['\nسوال\nآیا شما حیات جاودانی دارید؟\n\n\nجواب\nکتاب مقدس به وضوح راه رسیدن به زندگی جاودانه را به ما نشان داده است. نخست، ما باید این حقیقت را بپذیریم که بر ضّد خدا گناه کرده ایم: "زيرا همه گناه كرده\u200cاند و هيچكس به آن كمال مطلوب و پرجلالی كه خدا انتظار دارد، نرسيده است." (رومیان فصل 3 آیه 23). همۀ ما مرتکب اعمالی شده ایم که نه فقط خدا را خشنود نساخته، بلکه ما را سزاوار مجازات نیز نموده اند. از آنجا که همۀ گناهان ما در نهایت بر ضّد خدای ابدی صورت گرفته اند، ما شایستۀ مجازات ابدی هستیم. "زيرا هر كه گناه كند، تنها دستمزدی كه خواهد يافت، مرگ است؛ اما هر كه به خداوند ما عيسی مسيح ايمان آورد، پاداش او از خدا زندگی جاويد است." (رومیان فصل 6 آیه 23).\n\nاما عیسی مسیح، آن پسر ابدی خدا که بدون گناه بود (اول پطرس فصل 2 آیه 22)، انسان شد (یوحنا فصل 1 آیه های 1 و 14)، و مُرد تا جریمۀ گناهان ما را بپردازد. "اما ببينيد خدا چقدر ما را دوست داشت كه با وجود اينكه گناهكار بوديم، مسيح را فرستاد تا در راه ما فدا شود." (رومیان فصل 5 آیه 8). عیسی مسیح بر روی صلیب جان داد (یوحنا فصل 19 آیه های 31 تا 42

In [34]:
final[0:10]

['\nسوال\nآیا شما حیات جاودانی دارید؟\n\n\nجواب\nکتاب مقدس به وضوح راه رسیدن به زندگی جاودانه را به ما نشان داده است. نخست، ما باید این حقیقت را بپذیریم که بر ضّد خدا گناه کرده ایم: "زيرا همه گناه كرده\u200cاند و هيچكس به آن كمال مطلوب و پرجلالی كه خدا انتظار دارد، نرسيده است." (رومیان فصل 3 آیه 23). همۀ ما مرتکب اعمالی شده ایم که نه فقط خدا را خشنود نساخته، بلکه ما را سزاوار مجازات نیز نموده اند. از آنجا که همۀ گناهان ما در نهایت بر ضّد خدای ابدی صورت گرفته اند، ما شایستۀ مجازات ابدی هستیم. "زيرا هر كه گناه كند، تنها دستمزدی كه خواهد يافت، مرگ است؛ اما هر كه به خداوند ما عيسی مسيح ايمان آورد، پاداش او از خدا زندگی جاويد است." (رومیان فصل 6 آیه 23).\n\nاما عیسی مسیح، آن پسر ابدی خدا که بدون گناه بود (اول پطرس فصل 2 آیه 22)، انسان شد (یوحنا فصل 1 آیه های 1 و 14)، و مُرد تا جریمۀ گناهان ما را بپردازد. "اما ببينيد خدا چقدر ما را دوست داشت كه با وجود اينكه گناهكار بوديم، مسيح را فرستاد تا در راه ما فدا شود." (رومیان فصل 5 آیه 8). عیسی مسیح بر روی صلیب جان داد (یوحنا فصل 19 آیه های 31 تا 42

In [35]:
len(final)

492

In [36]:
a_tags22 = driver.find_elements(By.CSS_SELECTOR, ".content a")
print(len(a_tags22))

a_tags2=driver.find_elements(By.TAG_NAME,"a")
len(a_tags2)

32


52

In [37]:
with open("GodQuestion_Farsi.txt", "w", encoding="utf-8") as f:
    for item in final:
        f.write(item.strip() + "\n\n")